# Feature Selection, Model train, HP-finetune, 

In [11]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
%autoreload
from src.dataloader import *
from src.functions import *
from src.splitting import *
from src.dataset import *
from sklearn.ensemble import RandomForestRegressor

## Loading data

In [96]:
train_data = load_training_data("train_data")
X_train_a, y_train_a = train_data["train"]["a"]
X_train_b, y_train_b = train_data["train"]["b"]
X_train_c, y_train_c = train_data["train"]["c"]

## Feature selection

## TODO: Defining train-test split

## Training model

In [61]:
hyperparameters_a = {'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 10}
hyperparameters_b = {'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 10}
hyperparameters_c = {'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 10}

In [62]:
rf_a = RandomForestRegressor(**hyperparameters_a)
rf_a.fit(X_train_a.drop(columns = ["date_forecast"]), y_train_a["pv_measurement"])

RandomForestRegressor(min_samples_leaf=4, n_estimators=10)

In [63]:
rf_b = RandomForestRegressor(**hyperparameters_b)
rf_b.fit(X_train_b.drop(columns = ["date_forecast"]), y_train_b["pv_measurement"])

RandomForestRegressor(min_samples_leaf=4, n_estimators=10)

In [64]:
rf_b = RandomForestRegressor(**hyperparameters_c)
rf_b.fit(X_train_b.drop(columns = ["date_forecast"]), y_train_b["pv_measurement"])

RandomForestRegressor(min_samples_leaf=4, n_estimators=10)

## Tuning hyperparameters

In [122]:
param_grid = {
        'n_estimators': [50, 100, 200],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }
#hyperparameter_tuning(param_grid, X_train_a, y_train_a)

In [123]:
from src.dataset import Dataset
from src.splitting import SplitterMonth

data = Dataset(splitter=SplitterMonth(n_splits=5, shuffle=False))

In [124]:

            
data.preprocess(preprocess)

In [125]:
print(type(data.train_data["a"].split()[0]["train"].X))

<class 'pandas.core.frame.DataFrame'>


In [ ]:
from sklearn.ensemble import RandomForestRegressor

param_grid = {
        'n_estimators': [100, 200, 500, 1000],
        "criterion" : ["squared_error", "absolute_error"],
        'max_depth': [None, 2, 3, 5, 10, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
    }

rf = RandomForestRegressor()

hyperparameter_tuning(rf, param_grid, data.train_data["a"].split()[0]["train"])

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>


In [ ]:
#!git pull 
from src.dataset import Dataset
from src.splitting import SplitterMonth

data = Dataset(splitter=SplitterMonth(n_splits=5, shuffle=False))
def preprocess(data):
    
    times_y = set(data["train_targets"].time)
    
    for key in ["X_train_estimated", "X_train_observed"]:
            idx_X = data[key].date_forecast.isin(times_y).values
            data[key] = data[key][idx_X]
    return data
            
data.preprocess(preprocess)

def hyperparameter_tuning(est, param_grid, data):

    # Create the scorer
    scorer = make_scorer(mean_absolute_error, greater_is_better=False)

    # Create the grid search
    grid_search = GridSearchCV(
        estimator=est, param_grid=param_grid, scoring=scorer,
        cv = data.split_index(), verbose=2, n_jobs=-1
    )

    # Should be done in pipeline
    X = data.X.dropna(axis=1)
    X = X[[c for c in X.columns if not c in ['date_calc', 'date_forecast']]]
    
    # Fir the grid search
    grid_search.fit(X, data.y["pv_measurement"])

    # Get the best parameters
    best_params = grid_search.best_params_
    print(f"Best parameters b: {best_params} - {grid_search.best_score_}")
    return best_params

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import Lasso

grid = {
    "n_estimators": [1000], 
    "loss": ["absolute_error"],
    "max_depth": [10, 20, 30],
    "learning_rate": [0.02],
}
rf = GradientBoostingRegressor()
print("test")
hyperparameter_tuning_a = hyperparameter_tuning(rf, grid, data.train_data["a"].split()[0]["train"])
hyperparameter_tuning_b = hyperparameter_tuning(rf, grid, data.train_data["a"].split()[0]["train"])
hyperparameter_tuning_c = hyperparameter_tuning(rf, grid, data.train_data["a"].split()[0]["train"])

test
Fitting 5 folds for each of 3 candidates, totalling 15 fits
